# Packages and Functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import osmnx as ox
import geopandas as gpd
import rasterio
from rasterio.coords import BoundingBox
# import geowombat as gw
# from geowombat.data import l8_224078_20200518, l8_224078_20200518_B2, l8_224078_20200518_B3, l8_224078_20200518_B4, l8_224077_20200518_B2, l8_224077_20200518_B4

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Python and ML Learnings with AI COE\\Spatial Programming & Remote Sensing'

# Reading/Writing Remote Sensed Images

GeoWombat’s file opening is meant to mimic Xarray and Rasterio. That is, rasters are typically opened with a context manager using the function `geowombat.open`. GeoWombat uses `xarray.open_rasterio` to load data into an `xarray.DataArray`. In GeoWombat, the data are always chunked, meaning the data are always loaded as Dask arrays. As with `xarray.open_rasterio`, the opened DataArrays always have at least 1 band.

## Opening a single image

Opening an image with default settings looks similar to `xarray.open_rasterio` and `rasterio.open`. `geowombat.open` expects a file name (`str` or `pathlib.Path`).

In [2]:
# with gw.open(l8_224078_20200518) as src:
#     print(src)

In the example above, `src` is an `xarray.DataArray`. Thus, printing the object will display the underlying Dask array dimensions and chunks, the DataArray named coordinates, and the DataArray attributes.

It automatically converts the coordinates stored in `x` and `y`, and the different bands are stored in `band`. To select a single band we can simply select it with `src.sel(band = 1)`.

Let’s plot out what we have while removing missing values, stored at `0`, and switch the band order around to be RGB.

In [3]:
# fig, ax = plt.subplots(dpi = 200)
# with gw.open(l8_224078_20200518) as src:
#     src.where(src != 0).sel(band = [3, 2, 1]).gw.imshow(robust = True, ax = ax)
# plt.tight_layout(pad = 1)

## Opening multiple bands as a stack

Often, satellite bands will be stored in separate raster files. To open the files as one DataArray, specify a list instead of a file name.

In [4]:
# with gw.open([l8_224078_20200518_B2, l8_224078_20200518_B3, l8_224078_20200518_B4]) as src:
#     print(src)

By default, GeoWombat will stack multiple files by time. So, to stack multiple bands with the same timestamp, change the `stack_dim` keyword.

Also note the use of `band_names` parameter. Here we can set it to anything we want for instance `['blue','green','red']`.

In [5]:
# with gw.open(
#     [l8_224078_20200518_B2, l8_224078_20200518_B3, l8_224078_20200518_B4],
#     stack_dim = "band",
#     band_names = [1, 2, 3],
# ) as src:
#     print(src)

You will see this looks the same as the multiband raster:

In [6]:
# fig, ax = plt.subplots(dpi = 200)
# with gw.open(
#     [l8_224078_20200518_B2, l8_224078_20200518_B3, l8_224078_20200518_B4],
#     stack_dim = "band",
#     band_names = ['blue','green','red'],
# ) as src:
#     src.where(src != 0).sel(band = ['red','blue','green']).gw.imshow(robust = True, ax = ax)
# plt.tight_layout(pad = 1)

If time names are not specified with `stack_dim` = ‘time’, GeoWombat will attempt to parse dates from the file names. This could incur significant overhead when the file list is long. Therefore, it is good practice to specify the time names.

## Opening images from different sensors

One of many complications of using remotely sensed data is that there are so many different sensors such as LandSat, Sentinel, PlantScope etc each with their own band order and properties. Geowombat makes this much easier by providing a broad list of potential sensor configurations. [Read in more detail about sensor configurations here](https://pygis.io/docs/f_rs_config.html#f-rs-crs-sensors). For this section, let’s keep things simple and show you how to open a Sentinel 2 image using the configuration manager, frankly, it’s pretty easy:

In [7]:
# with gw.config.update(sensor = 's2'):
#     with gw.open('filepath.tif') as src:
#         print(src.band)

To see all available sensor names, use the **avail_sensors** property:

In [8]:
# with gw.open('filepath.tif') as src:
#     for sensor_name in src.gw.avail_sensors:
#         print(sensor_name)

## Opening multiple bands as a mosaic

When a list of files are given, GeoWombat will stack the data by default. To mosaic multiple files into the same band coordinate, use the **mosaic** keyword.

In [9]:
# with gw.open([l8_224077_20200518_B2, l8_224078_20200518_B2],
#               mosaic = True) as src:
#     print(src)

Now let's take a look at the mosaiced band 2 image values.

In [10]:
# fig, ax = plt.subplots(dpi = 200)
# with gw.open([l8_224077_20200518_B2, l8_224078_20200518_B2],
#               mosaic = True) as src:
#     src.where(src != 0).sel(band = 1).gw.imshow(robust = True, ax = ax)
# plt.tight_layout(pad = 1)

## Create a Time Series Stack

Let’s pretend for a moment that we have a time series of images from the same tile. We can stack them by passing a list of file names `[l8_224078_20200518, l8_224078_20200518]`, it also helps to be specific and assign `time_names = ['t1', 't2']`, and specify which dimension we want to stack our data along with `stack_dim = 'time'`.

In [11]:
# with gw.open([l8_224078_20200518, l8_224078_20200518],
#             band_names = ['blue', 'green', 'red'],
#             time_names = ['t1', 't2'],
#             stack_dim = 'time') as src:
#     print(src)

## Setting Missing Values

Many raster files do not have the missing value set properly in their profile. Geowombat makes it easy to set or update the missing data value using `nodata` in either `gw.open` or even in `gw.config.update` if you prefer.

In [12]:
# fig, ax = plt.subplots(dpi=200)
# with gw.open(l8_224078_20200518, nodata = 0) as src:
#     # replace 0 with nan
#     src = src.gw.mask_nodata()
#     src.sel(band = [3, 2, 1]).gw.imshow(robust = True, ax = ax)
# plt.tight_layout(pad = 1)

## Writing DataArrays to file

GeoWombat’s I/O can be accessed through the `to_vrt` and `to_raster` functions. These functions use Rasterio’s `write` and Dask.array `store` functions as I/O backends. In the examples below, src is an `xarray.DataArray` with the necessary transform information to write to an image file.

Write to a VRT file.

In [13]:
# # Transform the data to lat/lon
# with gw.config.update(ref_crs = 4326):

#     with gw.open(l8_224077_20200518_B4) as src:

#         # Write the data to a VRT
#         gw.to_vrt(src, 'lat_lon_file.vrt')

Write to a raster file.

In [14]:
# with gw.open(l8_224077_20200518_B4) as src:

#     # Xarray drops attributes
#     attrs = src.attrs.copy()

#     # Apply operations on the DataArray
#     src = src * 10.0

#     src.attrs = attrs

#     # Write the data to a GeoTiff
#     src.gw.save('output.tif',
#                 n_workers = 4,    # number of process workers sent to ``concurrent.futures``
#                 )   